## TYTAN tutorial 追加問題4（ナンプレ＆不等号ナンプレ）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）でナンプレ（数独）と不等号ナンプレを解く
](https://vigne-cla.com/21-19/)


### 問題
ナンバープレース（数独）の亜種である不等号ナンプレを解く。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_1-300x300.png" width = 18%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_2-300x300.png" width = 18%>
</div>

ブラウザ上で遊んでみると良い → [Puzzle Team](https://www.puzzle-futoshiki.com/futoshiki-4x4-easy/)


### このQUBO設定を使おう

<font color="red">One-hotエンコーディング</font>

例）1～3の自然数のどれかにする（＝3個の量子ビットから1個を１にする。1になった場所を自然数に割り当てる）
```
H = (q0 + q1 + q2 - 1)**2
```

<font color="red">「n個の量子ビットからm個を1にする」</font>

例）ある量子ビットを1にする（＝１個の量子ビットから１個を１にする）
```
H = (q0 - 1)**2
```

<font color="red">「2個の量子ビットが同時に1になったら報酬を与える」</font>

例）2個の量子ビットが同時に1になったら報酬を与える
```
H = -(q0 * q1)
```

盛りだくさんで難しい。

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

## 通常のナンプレの条件設定

4×4のナンプレでは4×4×4＝64個の量子ビットを用意する。4個セットで一つの数字を表すワンホット表現とし、次のように3次元的な配置を想像する。今回、混乱を避けるために量子ビット名を１始まりにしているが、かえって分かりにくいかもしれない。例えば、q112は1行目1列目のワンホット2番。これが１であればそのマスの数字は２となる。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_3-768x522.png" width = 55%>
</div>

まずワンホットの設定として、「各ワンホット(ｋの次元)は1つだけ1になる」を16回設定する。あとは、「各行(ｊの次元)は1つだけ1になる」を16回、「各列(ｉの次元)は1つだけ1になる」を16回設定すれば「各行、各列の数字が重複しない」となり通常のナンプレのルールを満たす。要するに、**3つの次元方向すべてについて「1つだけ1になる」**とする。

初めから記入されている数字については、その部分のワンホットのどこが1になるか指定しておく（「ある量子ビットを1にする」）。これで解が絞れる。

### 通常のナンプレのコード

左上マスを２で固定するだけの通常のナンプレを試す。

作者コメント：式が多いためfor文使ってしまってごめんなさい。printされた式を見て理解を深め、各自で便利関数を活用して美しく書いてください。

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [6]:
from tytan import *
import numpy as np

#量子ビットを用意する
#１始まり！！！！！！！！！！！！！！！
for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            command = f'q{i}{j}{k} = symbols(\'q{i}{j}{k}\')'
            print(command)
            exec(command)

#各マスはワンホットで一つだけ１になる（kの次元のワンホット）
H = 0
for i in range(1, 5):
    for j in range(1, 5):
        command = f'H += (q{i}{j}1 + q{i}{j}2 + q{i}{j}3 + q{i}{j}4 - 1)**2'
        print(command)
        exec(command)

#各行で数字が重複しない（つまり各行は一つだけ１になる）（jの次元のワンホット）
for i in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q{i}1{k} + q{i}2{k} + q{i}3{k} + q{i}4{k} - 1)**2'
        print(command)
        exec(command)

#各列で数字が重複しない（つまり各列は一つだけ１になる）（iの次元のワンホット）
for j in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q1{j}{k} + q2{j}{k} + q3{j}{k} + q4{j}{k} - 1)**2'
        print(command)
        exec(command)

#数字指定のマス
H += (q112 - 1)**2
print('H += (q112 - 1)**2')


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#上位3件をワンホットから整数に戻して確認
for r in result[:3]:
    print(r)

    box = np.array(list(r[0].values())).reshape(4, 4, 4)
    ans = np.zeros((4, 4), int)
    for i in range(4):
        for j in range(4):
            ans[i, j] = np.argmax(box[i, j, :]) + 1
    print(ans)

q111 = symbols('q111')
q112 = symbols('q112')
q113 = symbols('q113')
q114 = symbols('q114')
q121 = symbols('q121')
q122 = symbols('q122')
q123 = symbols('q123')
q124 = symbols('q124')
q131 = symbols('q131')
q132 = symbols('q132')
q133 = symbols('q133')
q134 = symbols('q134')
q141 = symbols('q141')
q142 = symbols('q142')
q143 = symbols('q143')
q144 = symbols('q144')
q211 = symbols('q211')
q212 = symbols('q212')
q213 = symbols('q213')
q214 = symbols('q214')
q221 = symbols('q221')
q222 = symbols('q222')
q223 = symbols('q223')
q224 = symbols('q224')
q231 = symbols('q231')
q232 = symbols('q232')
q233 = symbols('q233')
q234 = symbols('q234')
q241 = symbols('q241')
q242 = symbols('q242')
q243 = symbols('q243')
q244 = symbols('q244')
q311 = symbols('q311')
q312 = symbols('q312')
q313 = symbols('q313')
q314 = symbols('q314')
q321 = symbols('q321')
q322 = symbols('q322')
q323 = symbols('q323')
q324 = symbols('q324')
q331 = symbols('q331')
q332 = symbols('q332')
q333 = symbols('q333')
q334 = symb

左上の２を指定しただけなので非常に多くの解が得られる。たいていの問題はもっと数字が指定されていて、解が一意に定まるように作られている。それについては各自で試してほしい。

## 不等号ナンプレの条件設定

不等号ナンプレではさらに条件設定を追加する。不等号関係にある量子ビット８個に着目すると

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_4r-768x205.png" width = 65%>
</div>

小も大もワンホットなので1つだけ1が立つが、**大の方はより右側のビットが立たなければならない。**よって、「赤線のペアが同時に1になったときに報酬を与える」という設定でそれを促してやる。赤線のペアは6通りあるので6式になる。

（補足）ここまでの条件設定はすべて重み1.0で設定する。重みに差をつけるのは「優先したいこと」がある場合、つまり、叶わないものがある場合。今回はすべての設定が叶う解が存在することがわかっているので重みを付ける必要がない（むしろつけてしまうとおかしな事が起きる）。

### 不等号ナンプレのコード

条件式のprintはコメントアウトした。

In [5]:
from tytan import *
import numpy as np

#量子ビットを用意する
#１始まり！！！！！！！！！！！！！！！
for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            command = f'q{i}{j}{k} = symbols(\'q{i}{j}{k}\')'
            #print(command)
            exec(command)

#各マスはワンホットで一つだけ１になる（kの次元のワンホット）
H = 0
for i in range(1, 5):
    for j in range(1, 5):
        command = f'H += (q{i}{j}1 + q{i}{j}2 + q{i}{j}3 + q{i}{j}4 - 1)**2'
        #print(command)
        exec(command)

#各行で数字が重複しない（つまり各行は一つだけ１になる）（jの次元のワンホット）
for i in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q{i}1{k} + q{i}2{k} + q{i}3{k} + q{i}4{k} - 1)**2'
        #print(command)
        exec(command)

#各列で数字が重複しない（つまり各列は一つだけ１になる）（iの次元のワンホット）
for j in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q1{j}{k} + q2{j}{k} + q3{j}{k} + q4{j}{k} - 1)**2'
        #print(command)
        exec(command)

#数字指定のマス
H += (q112 - 1)**2
#print('H += (q112 - 1)**2')

#不等号
H += -(q121 * q222)
H += -(q121 * q223)
H += -(q121 * q224)
H += -(q122 * q223)
H += -(q122 * q224)
H += -(q123 * q224)

H += -(q141 * q242)
H += -(q141 * q243)
H += -(q141 * q244)
H += -(q142 * q243)
H += -(q142 * q244)
H += -(q143 * q244)

H += -(q311 * q212)
H += -(q311 * q213)
H += -(q311 * q214)
H += -(q312 * q213)
H += -(q312 * q214)
H += -(q313 * q214)

H += -(q321 * q332)
H += -(q321 * q333)
H += -(q321 * q334)
H += -(q322 * q333)
H += -(q322 * q334)
H += -(q323 * q334)


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=2000)

#上位3件をワンホットから整数に戻して確認
for r in result[:3]:
    print(r)

    box = np.array(list(r[0].values())).reshape(4, 4, 4)
    ans = np.zeros((4, 4), int)
    for i in range(4):
        for j in range(4):
            ans[i, j] = np.argmax(box[i, j, :]) + 1
    print(ans)

offset
49
[{'q111': 0, 'q112': 1, 'q113': 0, 'q114': 0, 'q121': 0, 'q122': 0, 'q123': 1, 'q124': 0, 'q131': 0, 'q132': 0, 'q133': 0, 'q134': 1, 'q141': 1, 'q142': 0, 'q143': 0, 'q144': 0, 'q211': 0, 'q212': 0, 'q213': 1, 'q214': 0, 'q221': 0, 'q222': 0, 'q223': 0, 'q224': 1, 'q231': 1, 'q232': 0, 'q233': 0, 'q234': 0, 'q241': 0, 'q242': 1, 'q243': 0, 'q244': 0, 'q311': 1, 'q312': 0, 'q313': 0, 'q314': 0, 'q321': 0, 'q322': 1, 'q323': 0, 'q324': 0, 'q331': 0, 'q332': 0, 'q333': 1, 'q334': 0, 'q341': 0, 'q342': 0, 'q343': 0, 'q344': 1, 'q411': 0, 'q412': 0, 'q413': 0, 'q414': 1, 'q421': 1, 'q422': 0, 'q423': 0, 'q424': 0, 'q431': 0, 'q432': 1, 'q433': 0, 'q434': 0, 'q441': 0, 'q442': 0, 'q443': 1, 'q444': 0}, -53.0, 1]
[[2 3 4 1]
 [3 4 1 2]
 [1 2 3 4]
 [4 1 2 3]]
[{'q111': 0, 'q112': 1, 'q113': 0, 'q114': 0, 'q121': 1, 'q122': 0, 'q123': 0, 'q124': 0, 'q131': 0, 'q132': 0, 'q133': 0, 'q134': 1, 'q141': 0, 'q142': 0, 'q143': 1, 'q144': 0, 'q211': 0, 'q212': 0, 'q213': 1, 'q214': 0, 'q221'

エネルギー＝-53が最適解であり、模範解答の通りである。（出る確率は低め）

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_2-300x300.png" width = 18%>
</div>